In [1]:
import os
import rootpath
import sys
sys.path.append(rootpath.detect())

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from analysis_utilities import PATH_TO_REPORT_REPO, PATH_TO_PROCESSED_DSAF_IGD, PATH_TO_PROCESSED_PAREGO_IGD
from analysis_utilities import PAREGO_IGD_LIST, DSAF_IGD_LIST
from analysis_utilities import load_result, save_table
from analysis_utilities import UNATTAINABLE_DSAF_IGD_LIST, UNATTAINABLE_PAREGO_IGD_LIST
from analysis_utilities import PARETO_DSAF_IGD_LIST, PARETO_PAREGO_IGD_LIST
from analysis_utilities import ATTAINABLE_DSAF_IGD_LIST, ATTAINABLE_PAREGO_IGD_LIST

In [8]:
def save_table(*args, **kwargs):
    pass
save_table(1, a=2)

In [4]:
TABLE_STAGES = [25, 50, 100, 150]

In [5]:
def apply_f_to_mask(df, mask, f):
    best_inds =  [*zip(*np.where(mask == 1))] 
    for i, j in best_inds:
        df.iloc[i, j] = f(df.iloc[i, j])

def format_table(df):
    df_str = df.astype(str)
    # italicise IQR columns
    mask_italics = np.zeros_like(df_str.values, dtype=bool)
    mask_italics[:,1::2] = True
    f_italics = lambda cell_value: r"\textit{"+cell_value+"}"
    apply_f_to_mask(df_str, mask_italics, f_italics)
    
    # get boolean mask of winners/same results for each
    values_directed = df_str.values[:,:int(df_str.values.shape[1]/2)]
    values_undirected = df_str.values[:,int(df_str.values.shape[1]/2):]
    
    # highlight best
    directed_wins = np.repeat((values_directed<values_undirected)[:, ::2], 2, axis=1)
    undirected_wins = np.repeat((values_directed>values_undirected)[:, ::2], 2, axis=1)
    mask_best = np.hstack([directed_wins, undirected_wins])
    f_best = lambda cell_value: r"\best "+cell_value
    apply_f_to_mask(df_str, mask_best, f_best)
    
    # highlight draws
    draw = np.repeat((values_directed==values_undirected)[:, ::2], 2, axis=1)
    mask_draw = np.hstack([draw, draw])
    f_draw = lambda cell_value: r"\statsimilar "+cell_value
    apply_f_to_mask(df_str, mask_draw, f_draw)

    return df_str

In [6]:
def get_med_iqr(file_name, file_dir):
    result = load_result(file_name, file_dir)
    stage_inds = [np.where(np.asarray(result.igd_hist_x)==ts)[0] for ts in TABLE_STAGES] 
    stage_inds = [s[0] for s in stage_inds] 
    
    med = np.median(result.igd_history[:, stage_inds], axis=0)
    lqr = np.quantile(result.igd_history[:, stage_inds], 0.25, axis=0)
    uqr = np.quantile(result.igd_history[:, stage_inds], 0.75, axis=0)
    return med, uqr-lqr


def get_table_dict_from_list(file_list, file_dir):
    D = {}
    for file_name in file_list:
        # get experiment name for table key
        key = file_name[:-7]
        
        med, iqr= get_med_iqr(file_name, file_dir)
        data0 = np.hstack((med.reshape(-1,1), iqr.reshape(-1,1)))
        if key+"_1" not in D.keys(): 
            D[key+f"_1"] = np.hstack([di for di in data0])#+[di for di in data1])
        else:
            D[key+f"_2"] = np.hstack([di for di in data0])#+[di for di in data1])
#         D[key] = np.hstack([di for di in data0])# +[dj for dj in data1])
    return D

In [7]:
def get_med_iqr_combined(file_name, file_dir):
    result = load_result(file_name, file_dir)
    stage_inds = [np.where(np.asarray(result.igd_hist_x)==ts)[0] for ts in TABLE_STAGES] 
    stage_inds  =[s[0] for s in stage_inds] 
    
    igd_history = result.igd_history[0]+result.igd_history[1]
    med = np.median(igd_history[:, stage_inds], axis=0)
    lqr = np.quantile(igd_history[:, stage_inds], 0.25, axis=0)
    uqr = np.quantile(igd_history[:, stage_inds], 0.75, axis=0)
    return med, uqr-lqr


def get_table_dict_from_list_combined(file_list, file_dir):
    D = {}
    for file_name in file_list:
        # get experiment name for table key
        key = file_name[:-7]
        
        med, iqr= get_med_iqr_combined(file_name, file_dir)
        data0 = np.hstack((med.reshape(-1,1), iqr.reshape(-1,1)))
        if key+"_1" not in D.keys(): 
            D[key+f"_1"] = np.hstack([di for di in data0])#+[di for di in data1])
        else:
            D[key+f"_2"] = np.hstack([di for di in data0])#+[di for di in data1])
#         D[key] = np.hstack([di for di in data0])# +[dj for dj in data1])
    return D

In [10]:
D_parego = get_table_dict_from_list(PAREGO_IGD_LIST, PATH_TO_PROCESSED_PAREGO_IGD)
D_dsaf = get_table_dict_from_list(DSAF_IGD_LIST, PATH_TO_PROCESSED_DSAF_IGD)

D_table_va = {k1: np.hstack([v1, v2]) for (k1, v1), (k2, v2) in zip(D_dsaf.items(), D_parego.items())}

{'wfg1_2obj_3dim_1': array([1.24575857, 0.1088038 , 1.22957074, 0.12213757, 1.22957074,
        0.12213757, 1.21966679, 0.11316959, 1.12119945, 0.05938434,
        1.10176343, 0.07011791, 1.08332582, 0.11349033, 1.08332582,
        0.11349033]),
 'wfg1_2obj_3dim_2': array([1.02749143, 0.08924051, 1.02058314, 0.07004115, 0.98073311,
        0.07395708, 0.98073311, 0.05485944, 1.05810786, 0.06276113,
        1.03066561, 0.04608639, 1.00097916, 0.0515384 , 0.99125997,
        0.07247235]),
 'wfg1_3obj_4dim_1': array([1.34524207, 0.29001189, 1.28513789, 0.23749127, 1.25669125,
        0.22291976, 1.25353059, 0.23943407, 1.30764913, 0.1900212 ,
        1.27431755, 0.07789339, 1.26392492, 0.06935082, 1.23393599,
        0.07592168]),
 'wfg1_3obj_4dim_2': array([1.49392625, 0.13702295, 1.44400184, 0.15708856, 1.44400184,
        0.13463507, 1.4412748 , 0.12309049, 1.67936082, 0.08862584,
        1.66329795, 0.08676576, 1.63867487, 0.09824897, 1.60189692,
        0.09294861]),
 'wfg1_4obj_5dim

In [9]:
D_parego = get_table_dict_from_list(PAREGO_IGD_LIST, PATH_TO_PROCESSED_PAREGO_IGD)
D_dsaf = get_table_dict_from_list(DSAF_IGD_LIST, PATH_TO_PROCESSED_DSAF_IGD)

D_table_vb = {k1: np.hstack([v1, v2]) for (k1, v1), (k2, v2) in zip(D_dsaf.items(), D_parego.items())}

In [10]:
D_parego = get_table_dict_from_list(ATTAINABLE_PAREGO_IGD_LIST, PATH_TO_PROCESSED_PAREGO_IGD)
D_dsaf = get_table_dict_from_list(ATTAINABLE_DSAF_IGD_LIST, PATH_TO_PROCESSED_DSAF_IGD)

D_table_attainable = {k1: np.hstack([v1, v2]) for (k1, v1), (k2, v2) in zip(D_dsaf.items(), D_parego.items())}

mic = pd.MultiIndex.from_product([["dsaf", "ParEgo"], [25, 50, 100, 150],['median', 'IQR']])
mir = pd.MultiIndex.from_product([[d[:-2].replace("_", " ") for d in D_table_attainable.keys()][::2], ["1", "2"]], names=[r"$f$", "Position"])
df_attainable = pd.DataFrame.from_dict(D_table_attainable, orient="index", columns=mic).round(3)
df_attainable.index = mir
df_attainable = format_table(df_attainable)
save_table(df_attainable, "attainable_parego_table_igd")
df_attainable

dsaf                                     \
                                  25                                 50    
                               median                   IQR       median   
$f$             Position                                                   
wfg1 2obj 3dim  1               1.301        \textit{0.147}        1.262   
                2         \best 1.027  \best \textit{0.089}  \best 1.021   
wfg1 3obj 4dim  1               1.404        \textit{0.358}        1.379   
                2         \best 1.494  \best \textit{0.137}  \best 1.444   
wfg1 4obj 5dim  1                1.84        \textit{0.181}        1.793   
                2         \best 1.937  \best \textit{0.171}  \best 1.893   
wfg2 2obj 6dim  1         \best 0.527  \best \textit{0.259}  \best 0.317   
                2                1.05        \textit{0.534}        1.028   
wfg2 3obj 6dim  1               1.714        \textit{0.876}        1.667   
                2                1.04        \textit{0.442}         1.01   
wfg2 4obj 10dim 1               2.393        \textit{0.509}        2.351   
                2               0.425        \textit{0.225}  \best 0.223   
wfg3 2obj 6dim  1         \best 0.206  \best \textit{0.096}   \best 0.08   
                2         \best 0.244  \best \textit{0.135}  \best 0.103   
wfg3 3obj 10dim 1         \best 0.478  \best \textit{0.295}  \best 0.193   
                2         \best 0.621  \best \textit{0.333}  \best 0.228   
wfg3 4obj 10dim 1         \best 0.506  \best \textit{0.239}  \best 0.176   
                2                 0.9        \textit{0.504}  \best 0.348   
wfg4 2obj 6dim  1               0.776        \textit{0.698}        0.503   
                2         \best 0.184  \best \textit{0.057}  \best 0.142   
wfg4 3obj 8dim  1               1.317        \textit{1.233}        0.834   
                2               1.079        \textit{1.445}        0.753   
wfg4 4obj 8dim  1               2.216        \textit{0.942}        0.943   
                2               1.569        \textit{1.158}  \best 0.733   
wfg5 2obj 6dim  1               0.464        \textit{0.431}        0.186   
                2           \best 0.2   \best \textit{0.19}  \best 0.092   
wfg5 3obj 8dim  1         \best 1.274  \best \textit{0.699}  \best 0.428   
                2               1.166        \textit{0.405}  \best 0.442   
wfg5 4obj 10dim 1         \best 2.624  \best \textit{0.839}        1.557   
                2               1.737         \textit{0.47}  \best 0.755   
wfg6 2obj 6dim  1               0.439        \textit{0.172}  \best 0.191   
                2         \best 0.496   \best \textit{0.24}  \best 0.298   
wfg6 3obj 8dim  1               1.172        \textit{0.723}  \best 0.403   
                2               0.786         \textit{0.38}  \best 0.406   
wfg6 4obj 10dim 1               2.333        \textit{0.928}        0.661   
                2               1.677         \textit{0.62}  \best 0.829   

                                                             \
                                                        100   
                                           IQR       median   
$f$             Position                                      
wfg1 2obj 3dim  1               \textit{0.199}        1.262   
                2          \best \textit{0.07}  \best 0.981   
wfg1 3obj 4dim  1               \textit{0.253}        1.369   
                2         \best \textit{0.157}  \best 1.444   
wfg1 4obj 5dim  1               \textit{0.165}        1.776   
                2         \best \textit{0.129}  \best 1.831   
wfg2 2obj 6dim  1         \best \textit{0.305}  \best 0.148   
                2               \textit{0.508}        0.893   
wfg2 3obj 6dim  1                \textit{0.87}        1.338   
                2                \textit{0.36}        0.833   
wfg2 4obj 10dim 1               \textit{0.632}         2.14   
                2         \best \textit{0.188} 

In [11]:
D_parego = get_table_dict_from_list(UNATTAINABLE_PAREGO_IGD_LIST, PATH_TO_PROCESSED_PAREGO_IGD)
D_dsaf = get_table_dict_from_list(UNATTAINABLE_DSAF_IGD_LIST, PATH_TO_PROCESSED_DSAF_IGD)

D_table_unattainable = {k1: np.hstack([v1, v2]) for (k1, v1), (k2, v2) in zip(D_dsaf.items(), D_parego.items())}

mic = pd.MultiIndex.from_product([["dsaf", "ParEgo"], [25, 50, 100, 150],['median', 'IQR']])
mir = pd.MultiIndex.from_product([[d[:-2].replace("_", " ") for d in D_table_unattainable.keys()][::2], ["1", "2"]], names=[r"$f$", "Position"])
df_unattainable = pd.DataFrame.from_dict(D_table_unattainable, orient="index", columns=mic).round(3)
df_unattainable.index = mir
df_unattainable = format_table(df_unattainable)
save_table(df_unattainable, "unattainable_parego_table_igd")
df_unattainable

dsaf                                     \
                                  25                                 50    
                               median                   IQR       median   
$f$             Position                                                   
wfg1 2obj 3dim  1               1.246        \textit{0.109}         1.23   
                2         \best 1.045  \best \textit{0.094}  \best 1.016   
wfg1 3obj 4dim  1               1.345         \textit{0.29}        1.285   
                2         \best 1.527   \best \textit{0.11}  \best 1.488   
wfg1 4obj 5dim  1               1.894        \textit{0.247}        1.823   
                2         \best 2.027  \best \textit{0.149}  \best 1.944   
wfg2 2obj 6dim  1                0.52        \textit{0.159}        0.411   
                2               0.808        \textit{0.341}        0.701   
wfg2 3obj 6dim  1                1.46        \textit{0.621}        1.408   
                2               0.861        \textit{0.245}        0.831   
wfg2 4obj 10dim 1                2.44        \textit{0.674}        2.318   
                2         \best 0.552  \best \textit{0.207}  \best 0.403   
wfg3 2obj 6dim  1         \best 0.233  \best \textit{0.137}   \best 0.09   
                2         \best 0.204  \best \textit{0.091}    \best 0.1   
wfg3 3obj 10dim 1         \best 0.412  \best \textit{0.166}  \best 0.222   
                2         \best 0.527  \best \textit{0.343}  \best 0.235   
wfg3 4obj 10dim 1               0.496        \textit{0.241}  \best 0.207   
                2               0.726        \textit{0.514}  \best 0.309   
wfg4 2obj 6dim  1               0.511        \textit{0.313}        0.395   
                2         \best 0.222  \best \textit{0.082}  \best 0.163   
wfg4 3obj 8dim  1         \best 0.575  \best \textit{0.371}  \best 0.376   
                2               0.994        \textit{0.801}        0.606   
wfg4 4obj 8dim  1               1.245        \textit{0.696}        0.501   
                2                1.26         \textit{0.57}  \best 0.597   
wfg5 2obj 6dim  1                0.31          \textit{0.2}        0.131   
                2         \best 0.178  \best \textit{0.195}  \best 0.106   
wfg5 3obj 8dim  1               0.766        \textit{0.317}  \best 0.346   
                2                0.69        \textit{0.379}   \best 0.27   
wfg5 4obj 10dim 1         \best 1.598  \best \textit{0.713}         0.85   
                2         \best 1.152  \best \textit{0.493}  \best 0.498   
wfg6 2obj 6dim  1         \best 0.347  \best \textit{0.131}   \best 0.15   
                2         \best 0.433  \best \textit{0.276}  \best 0.257   
wfg6 3obj 8dim  1               0.844        \textit{0.275}  \best 0.328   
                2         \best 0.522  \best \textit{0.292}  \best 0.265   
wfg6 4obj 10dim 1               1.504        \textit{0.606}        0.724   
                2         \best 1.157  \best \textit{0.495}  \best 0.818   

                                                             \
                                                        100   
                                           IQR       median   
$f$             Position                                      
wfg1 2obj 3dim  1               \textit{0.122}         1.23   
                2         \best \textit{0.086}        0.998   
wfg1 3obj 4dim  1               \textit{0.237}  \best 1.257   
                2         \best \textit{0.138}  \best 1.466   
wfg1 4obj 5dim  1               \textit{0.147}   \best 1.78   
                2          \best \textit{0.11}  \best 1.917   
wfg2 2obj 6dim  1               \textit{0.234}  \best 0.205   
                2               \textit{0.327}        0.625   
wfg2 3obj 6dim  1               \textit{0.748}        1.123   
                2               \textit{0.282}        0.685   
wfg2 4obj 10dim 1                \textit{0.88}        1.951   
                2         \best \textit{0.181} 

In [12]:
D_parego = get_table_dict_from_list(PARETO_PAREGO_IGD_LIST, PATH_TO_PROCESSED_PAREGO_IGD)
D_dsaf = get_table_dict_from_list(PARETO_DSAF_IGD_LIST, PATH_TO_PROCESSED_DSAF_IGD)

D_table_pareto = {k1: np.hstack([v1, v2]) for (k1, v1), (k2, v2) in zip(D_dsaf.items(), D_parego.items())}

mic = pd.MultiIndex.from_product([["dsaf", "ParEgo"], [25, 50, 100, 150],['median', 'IQR']])
mir = pd.MultiIndex.from_product([[d[:-2].replace("_", " ") for d in D_table_pareto.keys()][::2], ["1", "2"]], names=[r"$f$", "Position"])
df_pareto = pd.DataFrame.from_dict(D_table_pareto, orient="index", columns=mic).round(3)
df_pareto.index = mir
df_pareto = format_table(df_pareto)
save_table(df_pareto, "pareto_parego_table_igd")
df_pareto

dsaf                               \
                                         25                                 
                                      median                          IQR   
$f$             Position                                                    
wfg1 2obj 3dim  1                      1.297               \textit{0.135}   
                2                       1.05               \textit{0.076}   
wfg1 3obj 4dim  1                      1.372               \textit{0.095}   
                2                \best 1.531         \best \textit{0.182}   
wfg1 4obj 5dim  1                      1.906               \textit{0.247}   
                2                \best 2.019         \best \textit{0.126}   
wfg2 2obj 6dim  1         \statsimilar 0.496  \statsimilar \textit{0.294}   
                2                      0.892               \textit{0.374}   
wfg2 3obj 6dim  1                      1.529               \textit{0.558}   
                2                      0.841               \textit{0.372}   
wfg2 4obj 10dim 1                      2.597               \textit{0.575}   
                2                \best 0.457          \best \textit{0.28}   
wfg3 2obj 6dim  1                \best 0.214         \best \textit{0.139}   
                2                \best 0.208          \best \textit{0.11}   
wfg3 3obj 10dim 1                \best 0.484         \best \textit{0.171}   
                2                \best 0.531         \best \textit{0.263}   
wfg3 4obj 10dim 1                \best 0.496          \best \textit{0.31}   
                2                      0.946               \textit{0.378}   
wfg4 2obj 6dim  1                      0.543               \textit{0.347}   
                2                \best 0.209         \best \textit{0.073}   
wfg4 3obj 8dim  1                      0.759               \textit{0.883}   
                2                \best 0.903          \best \textit{0.82}   
wfg4 4obj 8dim  1                       2.17               \textit{1.956}   
                2                      1.282               \textit{0.465}   
wfg5 2obj 6dim  1                      0.307               \textit{0.219}   
                2                \best 0.209         \best \textit{0.251}   
wfg5 3obj 8dim  1                      0.974               \textit{0.462}   
                2                \best 0.632          \best \textit{0.29}   
wfg5 4obj 10dim 1                       2.69               \textit{1.115}   
                2                \best 1.422         \best \textit{0.471}   
wfg6 2obj 6dim  1                \best 0.303         \best \textit{0.246}   
                2                \best 0.539         \best \textit{0.328}   
wfg6 3obj 8dim  1                       0.99               \textit{0.519}   
                2                      0.588               \textit{0.308}   
wfg6 4obj 10dim 1                      2.036               \textit{0.957}   
                2                      1.558               \textit{0.358}   

                                                                          \
                                  50                                 100   
                               median                   IQR       median   
$f$             Position                                                   
wfg1 2obj 3dim  1               1.264        \textit{0.156}        1.255   
                2               1.016         \textit{0.06}  \best 0.983   
wfg1 3obj 4dim  1               1.341         \textit{0.08}        1.332   
                2         \best 1.485  \best \textit{0.198}  \best 1.466   
wfg1 4obj 5dim  1               1.822        \textit{0.183}        1.782   
                2          \best 1.94   \best \textit{0.18}  \best 1.919   
wfg2 2obj 6dim  1         \best 0.359  \best \textit{0.245}   \best 0.16   
                2               0.807        \textit{0.365}        0.711   
wfg2 3obj 6dim  1               1.358        \t

### RANKED SCORES

In [36]:
def get_comparison_matrices(resulta, resultb, intervals):
    print(intervals)
    assert isinstance(intervals, list), "intervals must be a list. For a single interval n supply [n]"
    assert all([i in resulta.igd_hist_x for i in intervals]), "Not all supplied intervals exist in results"
    assert all([i in resultb.igd_hist_x for i in intervals]), "Not all supplied intervals exist in results"
    
    indsa = [np.where(np.asarray(resulta.igd_hist_x)==i)[0][0] for i in intervals]
    indsb = [np.where(np.asarray(resultb.igd_hist_x)==i)[0][0] for i in intervals]
    print(resulta.igd_hist_x)
    print(indsa)
    print(indsb)
    ra = resulta.igd_history[:, indsa]
    rb = resultb.igd_history[:, indsb]
    
    Ma = (ra<rb)
    Mb = (rb<ra)
    M_draw = (ra==rb)
    return Ma, Mb, M_draw

def get_rank_table_from_list(f_names, result_list_a, result_list_b):
    D = {}
    for f_name, ra, rb in zip(f_names, result_list_a, result_list_b):
        Ma, Mb, M_draw = get_comparison_matrices(ra, rb, TABLE_STAGES)
        score_1 = Ma.astype(float)*1
        score_2 = M_draw.astype(float)*.5
        score = (score_1+score_2).mean(axis=0)
        D[f_name[:-5]] = score 
    return D

In [37]:
def add_gs_mask_to_df(df, mask):
    new_df = df.copy()
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            new_df.iloc[i, j] = r"\cellcolor[gray]{"+str(mask[i, j])+"} " +new_df.iloc[i, j]
    return new_df

def grayscale_df_mask(df):
    desired_max = 0.6
    scaled_values = df.values.copy()
    scaled_values[scaled_values<0.5] = 0.5
    
    scaled_values = (scaled_values-0.5)*2
    
    return (1-(scaled_values*desired_max)).round(3)

In [38]:
def add_gs_mask_to_df(df, mask):
    new_df = df.copy()
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            new_df.iloc[i, j] = r"\cellcolor[gray]{"+str(mask[i, j])+"} " +new_df.iloc[i, j]
    return new_df

def grayscale_df_mask(df):
    desired_max = 0.6
    scaled_values = df.values.copy()
    scaled_values[scaled_values<0.5] = 0.5
    
    scaled_values = (scaled_values-0.5)*2
    
    return (1-(scaled_values*desired_max)).round(3)

def format_rank_df(df):
    cols = df.columns
    new_cols = []
    for i in range(cols.shape[0]):
        new_cols.append(list(cols[i]))
        for j in range(np.shape(new_cols[i])[0]):
            if new_cols[i][j][0].lower() == "w":
                new_cols[i][j] = new_cols[i][j].upper() 
    
    df_new = pd.DataFrame(df.round(3).values, columns=pd.MultiIndex.from_tuples(new_cols), index=df.index)
#     df_new =df_new.rename(columns = {old: new for old, new in zip(cols, new_cols)}).round(3)
    
    # define win
    mask_win = (df_new.values>0.5)
    f_win = lambda value: "\\best "+value
    
    # define_grayscale
    gs_mask = grayscale_df_mask(df_new) 
    
    # define draw 
    mask_draw = (df_new.values==0.5)
    f_draw = lambda value: "\\statsimilar "+value
    
    # apply masks
    df_new = df_new.astype(str)
#     apply_f_to_mask(df_new, mask_win, f_win)
#     apply_f_to_mask(df_new, mask_draw, f_draw)
    return add_gs_mask_to_df(df_new, gs_mask)

In [39]:
dsaf_files = [load_result(fi, PATH_TO_PROCESSED_DSAF_IGD) for fi in DSAF_IGD_LIST]
parego_files = [load_result(fi, PATH_TO_PROCESSED_PAREGO_IGD) for fi in PAREGO_IGD_LIST]

for n1, n2 in zip(DSAF_IGD_LIST, PAREGO_IGD_LIST):
    assert n1 == n2
func_list = DSAF_IGD_LIST

In [40]:
pair = {"0": "0",
        "1": "1",
        "2": "2",
        "3": "0", 
        "4": "1", 
        "5": "2"}

f_names = [i.split("_")[0] for i in DSAF_IGD_LIST]
f_obj = [i.split("_")[1] for i in DSAF_IGD_LIST]
f_dim = [i.split("_")[2] for i in DSAF_IGD_LIST]
f_t = [i.split("_")[-1][:-5] for i in DSAF_IGD_LIST]
f_attain = [pair[i] for i in f_t]

D = get_rank_table_from_list(func_list, dsaf_files, parego_files)

df_rank_raw = pd.DataFrame.from_dict(D)
df_rank_raw.index = ["25", "50", "100", "150"]
df_rank_raw.columns = pd.MultiIndex.from_tuples(zip(f_names, f_obj, f_dim, f_t, f_attain))
df_rank_raw.columns.names = ["f_names", "f_obj", "f_dim", "f_t", "f_attain"]
df_rank_raw

[25, 50, 100, 150]
[ 10  10  15  20  25  30  35  40  45  50  55  60  65  70  75  80  85  90
  95 100 105 110 115 120 125 130 135 140 145 150]
[4, 9, 19, 29]
[1, 2, 3, 4]
[25, 50, 100, 150]
[ 10  10  15  20  25  30  35  40  45  50  55  60  65  70  75  80  85  90
  95 100 105 110 115 120 125 130 135 140 145 150]
[4, 9, 19, 29]
[1, 2, 3, 4]
[25, 50, 100, 150]
[ 10  10  15  20  25  30  35  40  45  50  55  60  65  70  75  80  85  90
  95 100 105 110 115 120 125 130 135 140 145 150]
[4, 9, 19, 29]
[1, 2, 3, 4]
[25, 50, 100, 150]
[ 10  10  15  20  25  30  35  40  45  50  55  60  65  70  75  80  85  90
  95 100 105 110 115 120 125 130 135 140 145 150]
[4, 9, 19, 29]
[1, 2, 3, 4]
[25, 50, 100, 150]
[ 10  10  15  20  25  30  35  40  45  50  55  60  65  70  75  80  85  90
  95 100 105 110 115 120 125 130 135 140 145 150]
[4, 9, 19, 29]
[1, 2, 3, 4]
[25, 50, 100, 150]
[ 10  10  15  20  25  30  35  40  45  50  55  60  65  70  75  80  85  90
  95 100 105 110 115 120 125 130 135 140 145 150]
[4, 9, 1

f_names       wfg1                                                    \
f_obj         2obj                                                     
f_dim         3dim                                                     
f_t              0         1         2         3         4         5   
f_attain         0         1         2         0         1         2   
25        0.161290  0.064516  0.193548  0.677419  0.387097  0.645161   
50        0.161290  0.032258  0.193548  0.580645  0.483871  0.548387   
100       0.096774  0.032258  0.161290  0.419355  0.451613  0.548387   
150       0.096774  0.064516  0.161290  0.387097  0.483871  0.516129   

f_names                                           ...      wfg6            \
f_obj         3obj                                ...      3obj             
f_dim         4dim                                ...      8dim             
f_t              0         1         2         3  ...         2         3   
f_attain         0         1         2         0  ...         2         0   
25        0.451613  0.451613  0.451613  0.709677  ...  0.483871  0.580645   
50        0.483871  0.354839  0.354839  0.645161  ...  0.645161  0.838710   
100       0.548387  0.387097  0.354839  0.709677  ...  0.903226  0.903226   
150       0.451613  0.387097  0.387097  0.709677  ...  1.000000  0.967742   

f_names                                                               \
f_obj                             4obj                                 
f_dim                            10dim                                 
f_t              4         5         0         1         2         3   
f_attain         1         2         0         1         2         0   
25        0.451613  0.419355  0.354839  0.354839  0.225806  0.645161   
50        0.806452  0.645161  0.387097  0.258065  0.225806  0.709677   
100       0.903226  0.967742  0.903226  0.741935  0.838710  0.870968   
150       1.000000  1.000000  0.935484  0.838710  0.935484  0.967742   

f_names                       
f_obj                         
f_dim                         
f_t              4         5  
f_attain         1         2  
25        0.548387  0.451613  
50        0.774194  0.677419  
100       0.903226  0.870968  
150       1.000000  0.935484  

[4 rows x 108 columns]

In [41]:
print("win %: ", np.mean(df_rank_raw.values))

win %:  0.5960648148148148


In [42]:
df_rank_overall = df_rank_raw.groupby(level=[0, 1, 2, 3], axis=1).mean()
df_rank_overall

f_names      wfg1                                                              \
f_obj        2obj                                                        3obj   
f_dim        3dim                                                        4dim   
f_t             0         1         2         3         4         5         0   
25       0.161290  0.064516  0.193548  0.677419  0.387097  0.645161  0.451613   
50       0.161290  0.032258  0.193548  0.580645  0.483871  0.548387  0.483871   
100      0.096774  0.032258  0.161290  0.419355  0.451613  0.548387  0.548387   
150      0.096774  0.064516  0.161290  0.387097  0.483871  0.516129  0.451613   

f_names                                ...      wfg6                      \
f_obj                                  ...      3obj                       
f_dim                                  ...      8dim                       
f_t             1         2         3  ...         2         3         4   
25       0.451613  0.451613  0.709677  ...  0.483871  0.580645  0.451613   
50       0.354839  0.354839  0.645161  ...  0.645161  0.838710  0.806452   
100      0.387097  0.354839  0.709677  ...  0.903226  0.903226  0.903226   
150      0.387097  0.387097  0.709677  ...  1.000000  0.967742  1.000000   

f_names                                                                        
f_obj                  4obj                                                    
f_dim                 10dim                                                    
f_t             5         0         1         2         3         4         5  
25       0.419355  0.354839  0.354839  0.225806  0.645161  0.548387  0.451613  
50       0.645161  0.387097  0.258065  0.225806  0.709677  0.774194  0.677419  
100      0.967742  0.903226  0.741935  0.838710  0.870968  0.903226  0.870968  
150      1.000000  0.935484  0.838710  0.935484  0.967742  1.000000  0.935484  

[4 rows x 108 columns]